### 1. 理论知识

1. ID3（Iterative Dichotomiser 3）是一种常用的决策树算法，主要用于**分类**任务。它通过构建树形模型来帮助决策。

2. **特征选择**： ID3 通过计算**信息增益**来选择最佳特征。**信息增益**是指使用某一特征进行分类，所带来的**不确定性减少程度**。ID3 首先计算整个数据集的熵，然后计算在每个特征下的条件熵，信息增益则是二者的差值。

3. **熵**： **熵**是衡量信息不确定性的一个指标。**熵越高，不确定性越大；熵越低，不确定性越小**。ID3 通过计算数据集中每个类别的分布来确定熵。

4. **递归构建**： ID3 采用递归方法来构建树。从根节点开始，选择信息增益最大的特征作为当前节点，然后根据该特征的不同取值分割数据集，继续为每个子集构建子树，直到满足停止条件（如所有样本属于同一类别或没有特征可用）

### 2.数据准备

##### 2.1 导入数据集

In [129]:
# 导入模块
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

# 处理数据集
iris=load_iris()
X = pd.DataFrame(data=iris.data, columns=iris.feature_names)
y = pd.DataFrame(data=iris.target, columns=["label"])

# 特征重命名
map_={
    'sepal length (cm)': '花萼长度',
    'sepal width (cm)': '花萼宽度',
    'petal length (cm)': '花瓣长度',
    'petal width (cm)': '花瓣宽度'
}
X=X.rename(columns=map_)
X

,花萼长度,花萼宽度,花瓣长度,花瓣宽度
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


#### 2.2 数据处理

1. ID3树要求特征是**离散**的，因此需要对Iris数据集进行**分箱**操作。

In [130]:
# 对每一列特征进行分箱
X_new = pd.DataFrame()
for c in X.columns:
    binned_data = pd.cut(X[c], bins=4, labels=[0, 1, 2, 3])
    X_new=pd.concat([X_new,binned_data],axis=1)
X_new

,花萼长度,花萼宽度,花瓣长度,花瓣宽度
0,0,2,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,2,0,0
...,...,...,...,...
145,2,1,2,3
146,2,0,2,2
147,2,1,2,3
148,2,2,2,3


### 3. 构建ID3树

#### 3.1 熵与信息增益

1. 信息熵的计算：在分类数据集中，信息熵用来衡量 数据集中**标签** 的 **纯度或不确定性**。具体来说，信息熵越高，表示数据集中标签的类别分布越分散，越不纯；信息熵越低，表示标签的类别越集中，数据集越纯。那么，如何计算信息熵，以及信息增益呢？

2. ![Example Image](img/信息熵1.jpg)

3. **将数据集根据某个特征分成若干个子集。假设特征 𝐴 有 𝑉 个可能取值。每个取值将数据集划分成一个子集。**

5. ![Example Image](img/子集信息熵1.jpg)

6. 最后，加权子集信息熵，计算条件熵与信息增益。

7. ![Example Image](img/条件熵与信息增益1.jpg)

1. 需要注意：**子集信息熵公式** = **原数据集信息熵公式**

2. 条件熵 = 子集的信息熵 * 子集比例，如对特征A有4个子集，每个子集的信息熵为[0.2, 0.3, 0.4, 0.1]，每个子集的比例[0.1, 0.5, 0.3, 0.1]

3. 则特征A的条件熵：即以特征A划分时，的子集的熵的加权和 = [0.2*0.1 + 0.3*0.5 + 0.4*0.3 + 0.1*0.1]

In [131]:
# sklearn中模型计算
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
model=DecisionTreeClassifier(criterion='entropy')
x_train,x_test,y_train,y_test=train_test_split(X_new,y)
model.fit(x_train, y_train)
y_pre=model.predict(x_test)
accuracy_score(y_test,y_pre)

0.8947368421052632